In [52]:
import os
import time
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys


In [3]:
CHROME_DRIVER_PATH = os.environ["CHROME_DRIVER_PATH"]

In [7]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("--headless")
webdriver_service = Service(CHROME_DRIVER_PATH)
driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)

## 네이버 웹툰

In [13]:
naver_webtoon_base_url = "https://comic.naver.com/"

In [96]:
titles = []
writers = []

for year in range(2005, 2023):

    url = naver_webtoon_base_url + f"webtoon/period?period={year}"

    driver.get(url)

    page_src = driver.page_source
    soup = BeautifulSoup(page_src, 'html.parser')

    title_in_year = [s.find("a").get_attribute_list("title")[0] for s in soup.find('ul', {"class": "img_list"}).find_all("dt")]
    writer_in_year = [s.text for s in soup.find_all('dd', {"class": "desc"})]

    titles.extend(title_in_year)
    writers.extend(writer_in_year)

    time.sleep(2 + np.random.randint(3))

In [97]:
len(titles) == len(writers)

True

In [98]:
len(titles)

4538

In [99]:
# len(writers)

4538

In [106]:
result = pd.DataFrame({"title":titles,
                    "writers":writers})

result = result.drop_duplicates()

In [143]:
result.head()

,title,writers
0,골방환상곡,\n워니 / 심윤수\n
1,사랑in,\n전세훈\n
3,마음의소리,\n조석\n
5,수사9단,\n김선권\n
6,와탕카,\n우주인\n


In [141]:
result.to_csv("./data/naver_webtoon.csv", index=False)

## 네이버 시리즈 - 장르소설

In [178]:
naver_series_base_url = "https://series.naver.com/"

In [179]:
total_cotents = 64402
contents_per_page = 25

In [180]:
page_n = int(np.ceil(total_cotents/contents_per_page))

In [181]:
titles = []
writers = []

for page in range(1, page_n + 1):

    url = naver_series_base_url + f"novel/categoryProductList.series?categoryTypeCode=all&page={page}"

    driver.get(url)
    time.sleep(0.5)

    page_src = driver.page_source
    soup = BeautifulSoup(page_src, 'html.parser')

    title_in_page = [s.find("div", {"class":"cont"}).find("a").get_attribute_list("title")[0] for s in soup.find("ul", {"class":"lst_list"}).find_all("li")]
    writer_in_page = [s.find("div", {"class":"cont"}).find("span", {"class":"ellipsis"}).text for s in soup.find("ul", {"class":"lst_list"}).find_all("li")]

    titles.extend(title_in_page)
    writers.extend(writer_in_page)

    time.sleep(np.random.exponential(0.5))

In [182]:
len(titles) == len(writers)

True

In [183]:
result = pd.DataFrame({"title":titles,
                    "writers":writers})

result = result.drop_duplicates()

In [184]:
result.to_csv("./data/naver_novel.csv", index=False)